In [0]:
!pip install -I pillow==5.1.0

In [0]:
# this sets up the virtual environment if gdrive doesn't contain the repo clone
# it by setting the the if to true and modifie the path as you wish
#@title Default title text
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
import os
!pip install -q xlrd
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ADL4CV')
#!ls
if False:
  from getpass import getpass
  user = getpass('GitHub user')
  password = getpass('GitHub password')
  os.environ['GitHub_AUTH'] = user + ':' + password
  !git clone https://$GitHub_AUTH@github.com/Cram13/ADL4CV_Project.git
os.chdir('ADL4CV_Project')
!git pull origin master
#!pip install -I pillow==5.1.0
!pip3 install torch torchvision
!pip3 install tensorboardX
#!pip install PIL
#!pip install image
#import PIL.image

In [0]:
 #!git checkout origin/master
!git pull origin master
#!git status
#!git fetch
#!git reset --hard origin/master

In [0]:
#setup options for training
from train_yolo_example import Opt
from yolo.yolo_net import Yolo
from yolo.loss import YoloLoss as yloss
from train_yolo_example import loadYoloBaseWeights
from yolo.yolo_utils import *

opt = Opt()
opt.batch_size = 10
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 1e-4
opt.num_workers = 4
anchor = [(6.88, 27.44), (11.93, 58.32), (19.90, 94.92), (40.00, 195.84), (97.96, 358.62)]
anchor[:] = [(x[0] / 32.0, x[1]/32.0) for x in anchor]
opt.model = Yolo(0, anchors=anchor)
loadYoloBaseWeights(opt.model, opt)
opt.criterion = yloss(opt.model.anchors, opt.reduction, filter_fkt=filter_non_zero_gt_without_id)
opt.log_path = './log/test_marc416'
opt.image_size = 416
opt.model.image_size = opt.image_size

In [0]:
path = './log/test_marc99/snapshot0004.tar'
device = next(opt.model.parameters()).device
opt.model.load_state_dict(torch.load(path, map_location=device)['model_state_dict'])
opt.optimizer = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999), weight_decay=opt.decay)
opt.optimizer.load_state_dict(torch.load(path, map_location=device)['optimizer_state_dict'])
#for param_group in opt.optimizer.param_groups:
#  param_group['lr'] = 1e-4
#opt.model.image_size = 832
#opt.image_size = 832

In [0]:
#fire up tensorboard in the background
#todo make path first
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format('./log/3_yolo_lstm_832/')#opt.log_path)
)
get_ipython().system_raw('/content/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:

import os
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from yolo.yolo_utils import *
from yolo.yolo_net import Yolo
from yolo.loss import YoloLoss as yloss
import torchvision
from tensorboardX import SummaryWriter
import shutil
import time
from dataset_utils.datasets.MOT_bb_singleframe import MOT_bb_singleframe
from dataset_utils.datasets.MOT_bb_singleframe import MOT_bb_singleframe_eval
import dataset_utils.MOT_utils as motu
from train_yolo_example import *
from torch.optim.lr_scheduler import StepLR, MultiStepLR
from dataset_utils.datasets.MotBBImageSingle import *
from torch.utils.data import DataLoader, Subset

def loadTrainEvalSet(opt):
    dataset = MotBBImageSingle('dataset_utils/Mot17_test_single.txt', use_only_first_video=False, seq_length=1, new_height=opt.image_size, new_width=opt.image_size)
    training_set = Subset(dataset, range(0, dataset.valid_begin))
    eval_set = Subset(dataset, range(dataset.valid_begin, len(dataset)))
    training_loader = DataLoader(training_set, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers,
                              drop_last=False)
    eval_loader = DataLoader(eval_set, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers,
                              drop_last=False)
    return training_set, training_loader, eval_set, eval_loader

def train(opt):
    # setup train and eval set
    if torch.cuda.is_available() and opt.useCuda:
        torch.cuda.manual_seed(123)
    else:
        torch.manual_seed(123)

    training_set, training_loader, eval_set, eval_loader = loadTrainEvalSet(opt)

    # log stuff
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)
    if torch.cuda.is_available() and opt.useCuda:
        writer.add_graph(opt.model.cpu(), torch.rand(opt.batch_size, 3, opt.image_size, opt.image_size))
        opt.model.cuda()
    else:
        writer.add_graph(opt.model, torch.rand(opt.batch_size, 3, opt.image_size, opt.image_size))

    # write the hyperparams lr, batchsize  and imagesize in the tensorboard file
    writer.add_text('Hyperparams',
                    'lr: {}, \nbatchsize: {}, \nimg_size:{}'.format(opt.learning_rate, opt.batch_size, opt.image_size))

    # loss and optimize
    if opt.optimizer is None:
        opt.optimizer = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),
                                         weight_decay=opt.decay)

    #scheduler = StepLR(opt.optimizer, step_size=1, gamma=0.75)
    #scheduler = MultiStepLR(opt.optimizer, milestones=[5, 30, 80], gamma=0.1)
    learning_rate_schedule = {"0": 1e-5, "5": 1e-4, "30": 1e-5, "50": 1e-6}

    epoch_len = len(training_loader)
    for epoch in range(opt.num_epoches):
        training_set.dataset.is_training = True
        print('num epoch: {:4d}'.format(epoch))
        #scheduler.step()
        
        #if str(epoch) in learning_rate_schedule.keys():
        #  for param_group in opt.optimizer.param_groups:
        #    param_group['lr'] = learning_rate_schedule[str(epoch)]
        opt.model.train()
        for img_nr, (gt, img) in enumerate(training_loader):
            if torch.cuda.is_available() and opt.useCuda:
                img = Variable(img.cuda(), requires_grad=True)
            else:
                img = Variable(img, requires_grad=True)
            opt.optimizer.zero_grad()
            logits = opt.model(img)
            loss, loss_coord, loss_conf = opt.criterion(logits, gt)
            writeLossToSummary(writer, 'Train', loss.item(),
                               loss_coord.item(), loss_conf.item(), epoch * epoch_len + img_nr)
            loss.backward()
            opt.optimizer.step()

        # eval stuff
        opt.model.eval()
        eval_set.dataset.is_training = False
        loss_ls = []
        loss_coord_ls = []
        loss_conf_ls = []
        all_ap = []
        all_ap1 = []
        for te_iter, te_batch in enumerate(eval_loader):
            te_label, te_image = te_batch
            num_sample = len(te_label)
            if torch.cuda.is_available() and opt.useCuda:
                te_image = te_image.cuda()
            with torch.no_grad():
                te_logits = opt.model(te_image)
                batch_loss, batch_loss_coord, batch_loss_conf = opt.criterion(te_logits, te_label)
                for i in range(num_sample):
                    ap = get_ap(te_logits[i], filter_non_zero_gt_without_id(te_label[i]), opt.image_size, opt.image_size, opt.model.anchors, .5)
                    ap1 = get_ap(te_logits[i], filter_non_zero_gt_without_id(te_label[i]), opt.image_size, opt.image_size, opt.model.anchors, .8)
                    if not np.isnan(ap):
                        all_ap.append(ap)
                    if not np.isnan(ap1):
                        all_ap1.append(ap1)
                # if te_iter % 10 ==0:
                #    img = np.array(draw_img(te_logits, te_image[0], opt.image_size, opt.model.anchors))
                #    print(img.shape)
                #    print(img.dtype)
                #    print(type(img))
                #    writer.add_image(f'Val/{epoch}', img)
            loss_ls.append(batch_loss * num_sample)
            loss_coord_ls.append(batch_loss_coord * num_sample)
            loss_conf_ls.append(batch_loss_conf * num_sample)
        te_loss = sum(loss_ls) / eval_set.__len__()
        te_coord_loss = sum(loss_coord_ls) / eval_set.__len__()
        te_conf_loss = sum(loss_conf_ls) / eval_set.__len__()
        #print('{}  {}   {}'.format(te_loss, te_coord_loss, te_conf_loss))
        writer.add_scalar('Val/AP0.5', np.mean(np.array(all_ap)), epoch * epoch_len)
        writer.add_scalar('Val/AP0.8', np.mean(np.array(all_ap1)), epoch * epoch_len)
        writeLossToSummary(writer, 'Val', te_loss.item(),
                           te_coord_loss.item(), te_conf_loss.item(), epoch * epoch_len)

        torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                    'optimizer_state_dict': opt.optimizer.state_dict()},
                   opt.log_path + '/snapshot{:04d}.tar'.format(epoch))
    writer.close()

In [0]:
train(opt)

In [0]:
from train_yolo_example import train
train(opt)

In [0]:
#fire up tensorboard in the background
#todo make path first
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format('./log')
)
get_ipython().system_raw('/content/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
import time
for i in range(3600):
  time.sleep(1)